This notebook is used for Capstone Project

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


Peer-Gradded Assignment 3

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
wikipage="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
results=requests.get(wikipage)
if results.status_code==200:
    soup=BeautifulSoup(results.content,"html.parser")
table=soup.find('table',{'class':'wikitable sortable'})
new_table=[]
for row in table.find_all('tr')[1:]:
    column_marker=0
    columns=row.find_all('td')
    new_table.append([column.get_text() for column in columns])
df=pd.DataFrame(new_table,columns=['Postal Code','Borough','Neighbourhood'])
df['Neighbourhood']=df['Neighbourhood'].str.replace('\n','')
df['Postal Code']=df['Postal Code'].str.replace('\n','')
df['Borough']=df['Borough'].str.replace('\n','')
df.drop(df[df['Borough']=="Not assigned"].index,inplace=True)
df.shape
for i in range(len(df)):
    if df.iloc[i,2]=="Not assigned":
        df.iloc[i,2]=df.iloc[i,1]
df.reset_index(drop=True)
df.sort_values(by=['Postal Code'],inplace=True)
df.reset_index(drop=True)


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


Sorting the geospatial data so that the postal codes match


In [2]:
import pandas as pd 
url='http://cocl.us/Geospatial_data'
af=pd.read_csv(url)
af.sort_values(by=['Postal Code'],inplace=True)
af

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Apply left join

In [3]:
df1=pd.merge(left=df,right=af,how='left',left_on='Postal Code',right_on='Postal Code')
df1.groupby(by=["Borough"])
df1



,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [4]:
pip install folium

     |████████████████████████████████| 93 kB 3.3 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
from geopy.geocoders import Nominatim
import folium
address='Toronto'
geolocator=Nominatim(user_agent="ny_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=False,
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
map_toronto
    

In [7]:
Dtorrento=df1[df1['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Dtorrento

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


In [8]:
from geopy.geocoders import Nominatim
import folium
address='Downtown Toronto'
geolocator=Nominatim(user_agent="ny_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(Dtorrento['Latitude'], Dtorrento['Longitude'], Dtorrento['Borough'], Dtorrento['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=False,
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
map_toronto
    

Define FourSquare Credentials

In [19]:
CLIENT_ID = 'QX3YBHILZT4VUZ2PW21TAGASAVTBYP1FRVOPEXRC3WYW3CHH' # your Foursquare ID
CLIENT_SECRET = 'IUANSDBH42S5CIODIO2UDYZEKWTVDAPQ5SZNQE32DI43U2VQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
t_venues = getNearbyVenues(names=Dtorrento['Neighbourhood'],
                                   latitudes=Dtorrento['Latitude'],
                                   longitudes=Dtorrento['Longitude']
                                  )
print(t_venues.shape)
t_venues
            

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government
(1232, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
...,...,...,...,...,...,...,...
1227,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.659415,-79.391221,Coffee Shop
1228,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,SUDS,43.659880,-79.394712,Bar
1229,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Convocation Hall,43.660828,-79.395245,College Auditorium
1230,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.658906,-79.388696,Coffee Shop


In [10]:
t_onehot = pd.get_dummies(t_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
t_onehot['Neighborhood'] = t_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [t_onehot.columns[-1]] + list(t_onehot.columns[:-1])
t_onehot = t_onehot[fixed_columns]
t_onehot

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
t_grouped = t_onehot.groupby('Neighborhood').mean().reset_index(drop=False)
t_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.017241,0.000000,0.000000,0.00,0.00,0.017241,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.015385,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.030769,0.000000,0.000000,0.00,0.00,0.015385,0.000000,0.000000,0.015385
3,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.025316,0.012658,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012658,...,0.00,0.012658,0.012658,0.012658,0.00,0.00,0.000000,0.000000,0.012658,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,...,0.01,0.020000,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.010000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,...,0.01,0.020000,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.000000,0.010000
7,"Garden District, Ryerson",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.01,0.010000,0.020000,0.000000,0.00,0.00,0.000000,0.010000,0.010000,0.010000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.01,0.000000,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.000000,0.010000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.063492,0.000000,0.047619,0.015873


In [74]:
from sklearn.cluster import KMeans
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torronto_grouped_clustering)
kmeans.labels_[0:19] 
#t_grouped.insert(0,'Cluster Labels',kmeans.labels_)
Dt_merged=t_grouped
Dt_merged

,Cluster Labels,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.00,0.017241,0.000000,0.000000,0.00,0.00,0.017241,0.000000,0.000000,0.000000
1,4,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,...,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
2,0,Central Bay Street,0.015385,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.00,0.030769,0.000000,0.000000,0.00,0.00,0.015385,0.000000,0.000000,0.015385
3,3,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
4,0,Church and Wellesley,0.025316,0.012658,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.00,0.012658,0.012658,0.012658,0.00,0.00,0.000000,0.000000,0.012658,0.000000
5,0,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.01,0.020000,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.010000
6,0,"First Canadian Place, Underground city",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.01,0.020000,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.000000,0.010000
7,0,"Garden District, Ryerson",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.01,0.010000,0.020000,0.000000,0.00,0.00,0.000000,0.010000,0.010000,0.010000
8,0,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.01,0.000000,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.000000,0.010000
9,1,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.063492,0.000000,0.047619,0.015873


In [35]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lon, poi in zip(Dt_merged['Latitude'], Dt_merged['Longitude'], Dt_merged['Neighbourhood'],):
    label = folium.Popup(str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=False,
        fill_color='orange',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [75]:

Dt_merged.loc[Dt_merged['Cluster Labels'] == 0, Dt_merged.columns[[1] + list(range(5, Dt_merged.shape[1]))]]

,Neighborhood,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.017241,...,0.00,0.017241,0.000000,0.000000,0.0,0.00,0.017241,0.00,0.000000,0.000000
2,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,...,0.00,0.030769,0.000000,0.000000,0.0,0.00,0.015385,0.00,0.000000,0.015385
4,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.012658,0.000000,0.00,0.000000,...,0.00,0.012658,0.012658,0.012658,0.0,0.00,0.000000,0.00,0.012658,0.000000
5,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0.0,0.0,0.040000,0.000000,0.00,0.010000,...,0.01,0.020000,0.000000,0.000000,0.0,0.00,0.020000,0.00,0.000000,0.010000
6,"First Canadian Place, Underground city",0.0,0.0,0.0,0.0,0.0,0.030000,0.000000,0.00,0.010000,...,0.01,0.020000,0.010000,0.000000,0.0,0.01,0.010000,0.00,0.000000,0.010000
7,"Garden District, Ryerson",0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.010000,...,0.01,0.010000,0.020000,0.000000,0.0,0.00,0.000000,0.01,0.010000,0.010000
8,"Harbourfront East, Union Station, Toronto Islands",0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.05,0.010000,...,0.01,0.000000,0.010000,0.000000,0.0,0.01,0.010000,0.00,0.000000,0.010000
10,"Queen's Park, Ontario Provincial Government",0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,...,0.00,0.000000,0.032258,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.000000
11,"Regent Park, Harbourfront",0.0,0.0,0.0,0.0,0.0,0.000000,0.021739,0.00,0.021739,...,0.00,0.000000,0.043478,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.000000
12,"Richmond, Adelaide, King",0.0,0.0,0.0,0.0,0.0,0.020619,0.000000,0.00,0.010309,...,0.00,0.030928,0.010309,0.000000,0.0,0.00,0.010309,0.00,0.000000,0.000000


In [76]:
Dt_merged.loc[Dt_merged['Cluster Labels'] == 1, Dt_merged.columns[[1] + list(range(5, Dt_merged.shape[1]))]]

,Neighborhood,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
9,"Kensington Market, Chinatown, Grange Park",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.063492,0.000000,0.047619,0.015873
18,"University of Toronto, Harbord",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.029412,0.0,0.0,0.0,0.000000,0.029412,0.000000,0.000000


In [77]:
Dt_merged.loc[Dt_merged['Cluster Labels'] == 2, Dt_merged.columns[[1] + list(range(5, Dt_merged.shape[1]))]]

,Neighborhood,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
13,Rosedale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0


In [78]:
Dt_merged.loc[Dt_merged['Cluster Labels'] == 3, Dt_merged.columns[[1] + list(range(5, Dt_merged.shape[1]))]]

,Neighborhood,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
3,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
Dt_merged.loc[Dt_merged['Cluster Labels'] == 4, Dt_merged.columns[[1] + list(range(5, Dt_merged.shape[1]))]]

,Neighborhood,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
Dt_merged.loc[Dt_merged['Cluster Labels'] == 5, Dt_merged.columns[[1] + list(range(5, Dt_merged.shape[1]))]]

,Neighborhood,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
